In [1]:
import time
import json
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# 1. THE MOCK BACKEND LOGIC
# In production, this would be an API call or Elasticsearch query
def search_engine(query):
    # Mock Database
    database = [
        {"id": 1, "type": "article", "title": "Optimizing SQL Queries", "tags": ["Database", "Performance"], "snippet": "Use EXPLAIN ANALYZE to understand query plans. Avoid SELECT * in production..."},
        {"id": 2, "type": "video", "title": "Intro to Kubernetes", "tags": ["DevOps", "K8s"], "snippet": "A 10-minute crash course on Pods, Services, and Ingress controllers..."},
        {"id": 3, "type": "doc", "title": "Python 3.11 Features", "tags": ["Python", "Updates"], "snippet": "Significant speed improvements and better error messages in tracebacks..."},
        {"id": 4, "type": "article", "title": "React Hooks Patterns", "tags": ["Frontend", "React"], "snippet": "Understanding useMemo and useCallback to prevent unnecessary re-renders..."},
    ]
    
    # Simulate Network Latency
    time.sleep(0.8) 
    
    # Search Logic
    if not query:
        return database
    
    query = query.lower()
    return [doc for doc in database if query in doc['title'].lower() or query in doc['snippet'].lower()]

# 2. THE STYLING (CSS)
# We inject this into the notebook to style our HTML output later
style = """
<style>
    :root {
        --primary: #2563eb;
        --bg-card: #ffffff;
        --text-main: #1e293b;
        --text-sub: #64748b;
        --border: #e2e8f0;
    }

    /* Container for results */
    .search-results-container {
        font-family: 'Segoe UI', sans-serif;
        max-width: 800px;
        margin-top: 20px;
    }

    /* Individual Result Card */
    .result-card {
        background: var(--bg-card);
        border: 1px solid var(--border);
        border-radius: 8px;
        padding: 16px;
        margin-bottom: 12px;
        transition: transform 0.1s ease, box-shadow 0.1s ease;
        border-left: 4px solid var(--primary);
    }
    
    .result-card:hover {
        transform: translateY(-2px);
        box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
    }

    /* Typography */
    .card-title {
        font-size: 1.1rem;
        font-weight: 600;
        color: var(--text-main);
        margin: 0 0 8px 0;
        cursor: pointer;
    }
    
    .card-snippet {
        font-size: 0.95rem;
        color: var(--text-sub);
        line-height: 1.5;
    }

    .card-tags {
        margin-top: 10px;
        display: flex;
        gap: 8px;
    }

    .tag {
        font-size: 0.75rem;
        background: #eff6ff;
        color: var(--primary);
        padding: 2px 8px;
        border-radius: 12px;
        font-weight: 500;
    }

    .meta-info {
        font-size: 0.75rem;
        color: #94a3b8;
        margin-bottom: 4px;
        text-transform: uppercase;
        letter-spacing: 0.05em;
    }
</style>
"""
display(HTML(style))

In [2]:
class SearchApp:
    def __init__(self):
        # -- UI Components --
        
        # 1. FIX: Add continuous_update=False
        # This prevents the 'value' from changing until you hit Enter or click away
        self.search_box = widgets.Text(
            placeholder='Try "Python" or "Database"...',
            description='',
            layout=widgets.Layout(width='70%', height='40px'),
            continuous_update=False 
        )
        
        self.search_btn = widgets.Button(
            description='Search',
            icon='search',
            button_style='primary',
            layout=widgets.Layout(width='15%', height='40px')
        )
        
        self.out = widgets.Output()
        
        self.container = widgets.VBox([
            widgets.HBox([self.search_box, self.search_btn], layout=widgets.Layout(justify_content='center', margin='0 0 20px 0')),
            self.out
        ])
        
        # -- Event Bindings --
        
        # 2. FIX: specific bindings for Button vs Text
        self.search_btn.on_click(self.on_search)
        
        # Observe the 'value' change. Because continuous_update is False,
        # this only triggers on Enter.
        self.search_box.observe(self.on_search, names='value')

    def render_results(self, results):
        html_content = '<div class="search-results-container">'
        
        if not results:
            html_content += """
                <div style="text-align: center; color: #64748b; padding: 40px;">
                    <h3>No documents found</h3>
                    <p>Try adjusting your search terms.</p>
                </div>
            """
        else:
            html_content += f'<div style="margin-bottom: 15px; color: #64748b;">Found {len(results)} results</div>'
            
            for doc in results:
                tags_html = "".join([f'<span class="tag">{t}</span>' for t in doc['tags']])
                
                html_content += f"""
                <div class="result-card">
                    <div class="meta-info">{doc['type']} • ID: {doc['id']}</div>
                    <div class="card-title">{doc['title']}</div>
                    <div class="card-snippet">{doc['snippet']}</div>
                    <div class="card-tags">{tags_html}</div>
                </div>
                """
        
        html_content += '</div>'
        return html_content

    # 3. FIX: Use *args to accept input from both Button (obj) and Observer (dict)
    def on_search(self, *args):
        query = self.search_box.value
        
        with self.out:
            clear_output(wait=True)
            display(HTML('<div style="text-align:center; padding: 20px;"><img src="https://cdnjs.cloudflare.com/ajax/libs/galleriffic/2.0.1/css/loader.gif" width="20"> Searching...</div>'))
            
            # Fetch Data
            results = search_engine(query)
            
            # Render Final UI
            clear_output(wait=True)
            display(HTML(self.render_results(results)))

    def display(self):
        display(self.container)

In [3]:
app = SearchApp()
app.display()